In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import sys
import timeit
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import asarray 
from datasets import load_dataset 
from sklearn import model_selection
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from vit_keras import vit
import warnings
from PIL import Image
import random

In [ ]:
#Elige una configuracion (CF1, CF2, CF3, CF4, CF5) y maquela como True
#######################
CF = 'CF1'
CF1_flag = True
CF2_flag = False
CF3_flag = False
CF4_flag = False
CF5_flag = False
#######################

#Especifique los directorios necesarios
#######################
path_or = '.\\Datos\\'
path_img = path_or + f'{CF}\\imagenes\\'
input_path = path_or + f'{CF}\\salidas\\'
new_images = path_or + f'{CF}\\new_images\\'
#######################

In [ ]:
#Elige si quieres crear el archivo de metadatos de tu configuracion elegida (debe llamarse 'metadata_CFX.csv', donde X es el numero, y estar vacio)
#######################
metadatos_CF1_edit_flag = False
metadatos_CF2_edit_flag = False
metadatos_CF3_edit_flag = False
metadatos_CF4_edit_flag = False
metadatos_CF5_edit_flag = False
#######################

#Elige si convertir en variable dataset todos los nombres y clases de las imagenes
#######################
leer_imagenes_flag = True
#######################

#Elige si realizar un Oversample
# * Las nuevas imagenes se guaradn en 'new_images'
# * Copiar las nuevas imagenes de 'new_images' a las carpetas 'imagenes/(clase)' correspondiente, y volver a generar metadata de 0.
#######################
#Numero total de imagenes por clase en orden. Por ejemplo: CF1 [274, 71, 41, 33, 407, 684, 307], CF2 [1136 ,681]
numero = [274, 71, 41, 33, 407, 684, 307]
#Numero objetivo para cada clase
numero_max = 684   
Oversample_flag = False
#######################

#Elige si guardar el resultado final en archivos .npy (y_train.npy, x_test.npy...)
#######################
redimensionar_datos_adhoc_flag = True #Para la version para Modelo 1 Adhoc
redimensionar_datos_224_flag = True
#######################

# Dimensiones de las imágenes para entrenar el modelo adhoc
WIDTH = 28
HEIGHT = 28

# Dimensiones de las imágenes para redes preentrenadas
WIDTH_224 = 224
HEIGHT_224 = 224

# Dimensiones de las imágenes para la representación visual
WIDTH_VIZ = 600
HEIGHT_VIZ = 600

In [ ]:
if CF1_flag or CF4_flag or CF5_flag:
    # Diccionario con las classes CF1
    classes = {0: ('bcc', 'CARCINOMA'),
               1: ('df', 'DERMATOFIBROMA'),
               2: ('bkl', 'LENTIGO'),
               3: ('vasc', 'LESION_VASCULAR'),
               4: ('mel', 'MELANOMA'),
               5: ('nv', 'NEVUS'),
               6: ('akiec', 'QUERATOSIS')}

    cm_plot_classes = ['bcc', 'df', 'bkl', 'vasc', 'mel', 'nv', 'akiec']

if CF2_flag or CF3_flag:
    # Diccionario con las classes CF2
    classes = {0: ('ben', 'BENIGNO'),
               1: ('mal', 'MALIGNO')}

    cm_plot_classes = ['ben', 'mal']

In [ ]:
def Oversample():
    # Configurar ImageDataGenerator
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True, 
        vertical_flip=True,
        fill_mode='nearest'
    )
    
    x2 = np.array(x)
    y2 = np.array(y)
    
    aux = 0
    # Número objetivo de imágenes después del aumento de datos
    objetivo_numero_imagenes = len(x)
    # Crear un array de NumPy para almacenar las imágenes originales
    x_os = list(x2)
    y_os = list(y2)
    for i in numero:
        aux2 = 0
        #if aux == 5: aux = aux + 1
        # Iterar hasta alcanzar el número objetivo de imágenes
        print(len(x_os))
        print(objetivo_numero_imagenes)
        print((objetivo_numero_imagenes+(numero_max-i)))
        while len(x_os) < (objetivo_numero_imagenes+(numero_max-i)):
            # Seleccionar una imagen aleatoria del array original
            imagen_original = random.choice(x2[y2 == aux])
            
            imagen_pil = Image.open(path_img+imagen_original)

            # Convertir a modo RGB si no está en ese modo
            imagen_pil_rgb = imagen_pil.convert("RGB")
            
            #imagen_original = load_img(imagen_original)
            #print((objetivo_numero_imagenes+(684-i)))
            #print(len(x_os))
            # Convertir la imagen Pillow a un array NumPy
            img_array = img_to_array(imagen_pil_rgb)
            img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión para el batch

            # Aplicar el generador
            #carpeta_destino = f'./Datos/CF1_aumented/'
            #ruta_guardado = os.path.join(carpeta_destino, f'{aux}')
            #augmented_images = datagen.flow(img_array, batch_size=1, save_to_dir=f'./Datos/CF1_aumented/{aux}/', save_prefix=' ', save_format='jpg')
            augmented_images = datagen.flow(img_array, batch_size=1)
            #augmented_images.next()
            # Convertir la imagen aumentada de nuevo a formato Pillow
            augmented_image_pil = Image.fromarray(np.uint8(augmented_images[0][0]))

            # Agregar la imagen aumentada al array
            x_os.append(augmented_image_pil)
            y_os.append(aux)

            carpeta_destino = new_images + f'{aux}/'
            os.makedirs(carpeta_destino, exist_ok=True)

            ruta_guardado = os.path.join(carpeta_destino, f'imagen_aumented_{aux}_{aux2}.png')
            aux2 = aux2 + 1
            array_to_img(augmented_image_pil).save(ruta_guardado)
        aux = aux + 1
        objetivo_numero_imagenes = len(x_os)

In [ ]:
from PIL import Image
import csv

#Generador metadatos CF1
if metadatos_CF1_edit_flag == True:
    data=[]
    with open(path_or + 'metadata_CF1.csv', 'w', newline='') as writeFile:
        writer = csv.writer(writeFile, delimiter=';')
        writer.writerow(['image', 'label'])
        for filename in os.listdir('./Datos/CF1/imagenes/CARCINOMA'):
            writer.writerow(['CARCINOMA/' + filename, '0'])
        for filename in os.listdir('./Datos/CF1/imagenes/DERMATOFIBROMA'):
            writer.writerow(['DERMATOFIBROMA/' + filename, '1'])
        for filename in os.listdir('./Datos/CF1/imagenes/LENTIGO'):
            writer.writerow(['LENTIGO/' + filename, '2'])
        for filename in os.listdir('./Datos/CF1/imagenes/LESION_VASCULAR'):
            writer.writerow(['LESION_VASCULAR/' + filename, '3'])
        for filename in os.listdir('./Datos/CF1/imagenes/MELANOMA'):
            writer.writerow(['MELANOMA/' + filename, '4'])
        for filename in os.listdir('./Datos/CF1/imagenes/NEVUS'):
            writer.writerow(['NEVUS/' + filename, '5'])
        for filename in os.listdir('./Datos/CF1/imagenes/QUERATOSIS'):
            writer.writerow(['QUERATOSIS/' + filename, '6'])
    writeFile.close()

In [ ]:
#Generador metadatos CF2
if metadatos_CF2_edit_flag == True:
    data=[]
    with open(path_or + 'metadata_CF2.csv', 'w', newline='') as writeFile:
        writer = csv.writer(writeFile, delimiter=';')
        writer.writerow(['image', 'label'])
        for filename in os.listdir('./Datos/CF2/imagenes/BENIGNO/'):
            writer.writerow(['BENIGNO/' + filename, '0'])
        for filename in os.listdir('./Datos/CF2/imagenes/MALIGNO/'):
            writer.writerow(['MALIGNO/' + filename, '1'])
        writeFile.close()

In [ ]:
#Generador metadatos CF3
if metadatos_CF3_edit_flag == True:
    data=[]
    with open(path_or + 'metadata_CF3.csv', 'w', newline='') as writeFile:
        writer = csv.writer(writeFile, delimiter=';')
        writer.writerow(['image', 'label'])
        for filename in os.listdir('./Datos/CF3/imagenes/BENIGNO'):
            writer.writerow(['BENIGNO/' + filename, '0'])
        for filename in os.listdir('./Datos/CF3/imagenes/MALIGNO'):
            writer.writerow(['MALIGNO/' + filename, '1'])
        writeFile.close()

In [ ]:
#Generador metadatos CF4
if metadatos_CF4_edit_flag == True:
    data=[]
    with open(path_or + 'metadata_CF4.csv', 'w', newline='') as writeFile:
        writer = csv.writer(writeFile, delimiter=';')
        writer.writerow(['image', 'label'])
        for filename in os.listdir('./Datos/CF4/imagenes/CARCINOMA'):
            writer.writerow(['CARCINOMA/' + filename, '0'])
        for filename in os.listdir('./Datos/CF4/imagenes/DERMATOFIBROMA'):
            writer.writerow(['DERMATOFIBROMA/' + filename, '1'])
        for filename in os.listdir('./Datos/CF4/imagenes/LENTIGO'):
            writer.writerow(['LENTIGO/' + filename, '2'])
        for filename in os.listdir('./Datos/CF4/imagenes/LESION_VASCULAR'):
            writer.writerow(['LESION_VASCULAR/' + filename, '3'])
        for filename in os.listdir('./Datos/CF4/imagenes/MELANOMA'):
            writer.writerow(['MELANOMA/' + filename, '4'])
        for filename in os.listdir('./Datos/CF4/imagenes/NEVUS'):
            writer.writerow(['NEVUS/' + filename, '5'])
        for filename in os.listdir('./Datos/CF4/imagenes/QUERATOSIS'):
            writer.writerow(['QUERATOSIS/' + filename, '6'])
    writeFile.close()

In [ ]:
#Generador metadatos CF5
if metadatos_CF5_edit_flag == True:
    data=[]
    with open(path_or + 'metadata_CF5.csv', 'w', newline='') as writeFile:
        writer = csv.writer(writeFile, delimiter=';')
        writer.writerow(['image', 'label'])
        for filename in os.listdir('./Datos/CF5/imagenes/CARCINOMA'):
            writer.writerow(['CARCINOMA/' + filename, '0'])
        for filename in os.listdir('./Datos/CF5/imagenes/DERMATOFIBROMA'):
            writer.writerow(['DERMATOFIBROMA/' + filename, '1'])
        for filename in os.listdir('./Datos/CF5/imagenes/LENTIGO'):
            writer.writerow(['LENTIGO/' + filename, '2'])
        for filename in os.listdir('./Datos/CF5/imagenes/LESION_VASCULAR'):
            writer.writerow(['LESION_VASCULAR/' + filename, '3'])
        for filename in os.listdir('./Datos/CF5/imagenes/MELANOMA'):
            writer.writerow(['MELANOMA/' + filename, '4'])
        for filename in os.listdir('./Datos/CF5/imagenes/NEVUS'):
            writer.writerow(['NEVUS/' + filename, '5'])
        for filename in os.listdir('./Datos/CF5/imagenes/QUERATOSIS'):
            writer.writerow(['QUERATOSIS/' + filename, '6'])
    writeFile.close()

In [ ]:
metadata = pd.read_csv(path_or + f'metadata_{CF}.csv', delimiter=';')
# Distribución de frecuencia de imágenes de cada clase
plt.figure(figsize = (12, 8))
sns.set_theme(style = 'darkgrid')
sns.countplot(data = metadata, x = 'label', order = metadata['label'].value_counts().index, palette = 'mako')
plt.xlabel('Clase', size = 15)
plt.ylabel('Cantidad', size = 15)
plt.title(f'Distribución de frecuencia de imágenes de cada clase en el {CF}', size = 20)

In [ ]:
metadata['label'].value_counts()

In [ ]:
metadata

In [ ]:
if leer_imagenes_flag == True:
    dataset = load_dataset('csv', data_files=[path_or+f'metadata_{CF}.csv'], delimiter=';')#data_dir=CF1_path)
    y = dataset['train']['label']
    x = dataset['train']['image']
    


In [ ]:
if Oversample_flag == True:
    Oversample() 

In [ ]:
def modificar_resolucion_imagenes(x, conjunto_nuevo, anchura, altura):
    resolucion = (anchura, altura)
    for imagen in x:
        imagen_pil = Image.open(path_img+imagen)
        imagen2 = imagen_pil.convert("RGB")
        imagen_redimensionada = imagen2.resize(resolucion)
        conjunto_nuevo.append(imagen_redimensionada)

In [ ]:
if redimensionar_datos_adhoc_flag == True:
    x_adhoc = []
    modificar_resolucion_imagenes(x, x_adhoc, WIDTH, HEIGHT)

    x_adhoc_nor = []

    for imagen in x_adhoc:
        imagen2 = imagen.convert("RGB")
        # Convertir la imagen a un arreglo NumPy
        img_array = np.array(imagen2)
        x_adhoc_nor.append(img_array)


    x_adhoc_nor = (x_adhoc_nor - np.mean(x_adhoc_nor)) / np.std(x_adhoc_nor)

    x_adhoc_nor = np.array(x_adhoc_nor)
    x_adhoc_nor = x_adhoc_nor.reshape(-1, WIDTH, HEIGHT, 3)
    y = np.array(y)

    x_train, x_test, y_train, y_test = model_selection.train_test_split(x_adhoc_nor, y, test_size=0.2, random_state=1)
    x_test, x_val, y_test, y_val = model_selection.train_test_split(x_test, y_test, test_size=0.5, random_state=1)

    # Conversión a ndarrays de números reales
    x_train = np.asarray(x_train).astype(np.float16)
    np.save(input_path + 'x_train_adhoc', x_train)

    x_test = np.asarray(x_test).astype(np.float16)
    np.save(input_path + 'x_test_adhoc', x_test)
    
    x_val = np.asarray(x_val).astype(np.float16)
    np.save(input_path + 'x_val_adhoc', x_val)
    
    if CF1_flag:
        y_train = np.asarray(y_train).astype(np.float16)
        y_test = np.asarray(y_test).astype(np.float16)
        y_val = np.asarray(y_val).astype(np.float16)

    if CF2_flag or CF3_flag:
        y_train = np.asarray(y_train).astype(bool)
        y_test = np.asarray(y_test).astype(bool)
        y_val = np.asarray(y_val).astype(bool)

    np.save(input_path + 'y_train_adhoc', y_train)
    np.save(input_path + 'y_test_adhoc', y_test)
    np.save(input_path + 'y_val_adhoc', y_val)

In [ ]:
if redimensionar_datos_224_flag == True:
    x2 = []
    modificar_resolucion_imagenes(x, x2, WIDTH_224, HEIGHT_224)

    x2_nor = []
    x2_nor = []
    for imagen in x2:
        imagen2 = imagen.convert("RGB")
        # Convertir la imagen a un arreglo NumPy
        img_array = np.array(imagen2)
        x2_nor.append(img_array)
    x2_nor = (x2_nor - np.mean(x2_nor)) / np.std(x2_nor)

    x2_nor = np.array(x2_nor)
    x2_nor = x2_nor.reshape(-1, WIDTH_224, HEIGHT_224, 3)
    y = np.array(y)

    x_train, x_test, y_train, y_test = model_selection.train_test_split(x2_nor, y, test_size=0.2, random_state=1)
    x_test, x_val, y_test, y_val = model_selection.train_test_split(x_test, y_test, test_size=0.5, random_state=1)

    # Conversión a ndarrays de números reales
    x_train = np.asarray(x_train).astype(np.float16)
    np.save(input_path + 'x_train', x_train)

    x_test = np.asarray(x_test).astype(np.float16)
    np.save(input_path + 'x_test', x_test)
    
    x_val = np.asarray(x_val).astype(np.float16)
    np.save(input_path + 'x_val', x_val)
    
    if CF1_flag:
        y_train = np.asarray(y_train).astype(np.float16)
        y_test = np.asarray(y_test).astype(np.float16)
        y_val = np.asarray(y_val).astype(np.float16)

    if CF2_flag or CF3_flag:
        y_train = np.asarray(y_train).astype(bool)
        y_test = np.asarray(y_test).astype(bool)
        y_val = np.asarray(y_val).astype(bool)

    np.save(input_path + 'y_train', y_train)
    np.save(input_path + 'y_test', y_test)
    np.save(input_path + 'y_val', y_val)